In [2]:
from datasets import load_dataset
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, default_collate
import matplotlib.pyplot as plt
import matplotlib as mpl
import torchvision.transforms.functional as TF
from miniai.datasets import *
from operator import itemgetter
from miniai.conv import to_device

In [3]:
torch.manual_seed(42)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
mpl.rcParams['image.cmap']='gray'

In [4]:
ds = load_dataset ('fashion_mnist')
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [5]:
type(ds['train']['image'][0]), type(ds['train']['label'][0])


(PIL.PngImagePlugin.PngImageFile, int)

In [6]:
def trans_pil_tensor(b):
    b['image'] = [TF.pil_to_tensor(o) for o in b['image']]
    return b

In [7]:
ds_transform = ds.with_transform(trans_pil_tensor)

In [8]:
ds_transform['train']['image'][0].shape

torch.Size([1, 28, 28])

In [9]:
train_ds = ds_transform['train']
valid_ds = ds_transform['test']
x, y = train_ds.features
x,y

('image', 'label')

In [10]:
bs = 250

In [11]:
def _get_b(ds):
    ig = itemgetter(*ds.features)
    def _func(b): 
        return ig(default_collate(b))
    return _func

cf = _get_b(train_ds)

In [12]:
def collate_device(b):
    return to_device(cf(b))

In [20]:
def dls():
    return { k: DataLoader(v, batch_size=bs, collate_fn=collate_device) for k,v in ds_transform.items()}

In [21]:
dls = dls()
dl_train = dls['train']
dl_valid = dls['test']

In [33]:
y_lbl = ds['train'].features['label']
y_lbl

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [37]:
y_lbl.str2int(yb[:4])

ValueError: Invalid string class label tensor(9, device='cuda:0')

In [25]:
get_feat = itemgetter(y_lbl)

In [26]:
xb, yb = next(iter(dl_train))

In [31]:
get_feat(yb[0].item())

TypeError: 'int' object is not subscriptable

In [2]:
names = "Top Trouser Pullover Dress Coat Sandal Shirt Sneaker Bag Boot".split()
names

['Top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Boot']